## ANN - DATA PREPROCESSING, ENGINEERING (delete unecessary I/P features, make all values of data numerical (encoders), split IP/OP data, split total data into train & test, scaling train & test I/P data, save pickle files of encoders & scaler)


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split #to divide ddataset into train & test data 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
#Load the dataset

data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Preprocess data
##Drop irrelevant features (row number, customerid, surname)

data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Encode Categorical Variables

##Gender column encoding using Label Encoder
label_encoder_gender = LabelEncoder() #used for binary values case ex: male/female
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

#Geography column encoding using OHE 
#With Label Encoder we get 0,1,2,3.. values assigned to non-binary values of a column, but ML will read it as 
#priority sequence, so better to use OHE for non-binary values in a column case

onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [5]:
onehot_encoder_geo.get_feature_names_out(['Geography']) #extract different values in Geography column

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [6]:
#Create a table/dataframe with value names (columns) extracted above from Geography column and give OHE values
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
#Add OHE geography columns to dataset and drop original Geography column

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
#Save LabelEncoder & OHE in pickle files

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [9]:
#Divide dataset into indep (i/p) and dep (o/p) features
X = data.drop('Exited',axis=1)
y = data['Exited']

#Train and test data split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#Scale the indep (i/p) features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]], shape=(8000, 12))

In [11]:
#Save scaler model as pickle file

with open ('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## DATA IS READY, NOW TRAIN ANN 

In [12]:
import sys
print(sys.executable)

c:\Users\sande\OneDrive\Desktop\DS ML NLP Krish Naik\Exercises\Practise NLP\NLP DL\ANN\.venv\Scripts\python.exe


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.20.0


In [14]:
#Build ANN model

model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1 connected with input layer
    Dense(32,activation='relu'), #HL2 automatically connected to HL1 as it is 'Sequential' model
    Dense(1,activation='sigmoid') #O/P layer connected to HL2
])

c:\Users\sande\OneDrive\Desktop\DS ML NLP Krish Naik\Exercises\Practise NLP\NLP DL\ANN\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary() #gives total parameters weights & biases

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01) #define optimizer and loss function
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [17]:
#To do F/W & B/W propogation, we should compile the model

model.compile(optimizer=opt,loss=loss,metrics=['accuracy']) #compile model by defining its optimizer and loss fn

In [18]:
#Setup Tensorboard to capture training logs and display them

log_dir = "logs/fit/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1) #helps to visualize training logs (better than matplotlib manual way)
#histogram_freq = 1 means log for every 1 epoch

In [19]:
#Setup Early Stopping to limit number of epochs (cycles) of training based on loss fn value decreasing or not

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True) 
#Sets patience of 5 epochs before early stopping and before stopping, last best weights are restored


In [20]:
#Train the model

history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                    callbacks=[tensorflow_callback,early_stopping_callback]
                    )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8382 - loss: 0.3857 - val_accuracy: 0.8595 - val_loss: 0.3530
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8566 - loss: 0.3536 - val_accuracy: 0.8600 - val_loss: 0.3427
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8589 - loss: 0.3476 - val_accuracy: 0.8620 - val_loss: 0.3387
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8609 - loss: 0.3441 - val_accuracy: 0.8590 - val_loss: 0.3422
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8597 - loss: 0.3407 - val_accuracy: 0.8665 - val_loss: 0.3405
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8619 - loss: 0.3377 - val_accuracy: 0.8620 - val_loss: 0.3386
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8606 - loss: 0.3359 - val_accuracy: 0.8590 - val_loss: 0.3471
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8635 - loss: 0.3355 - val_accu

In [21]:
#Save model file as .h5 file as it is compatible with Keras

model.save('model.h5')


In [22]:
#Load Tensorboard extension

%load_ext tensorboard

In [28]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6008 (pid 19748), started 19:52:18 ago. (Use '!kill 19748' to kill it.)

In [24]:
#Load the pickle file to do predictions in other code file



